In [14]:
import pandas as pd

# Load E*TRADE data
df = pd.read_csv("ByStatus_expanded.csv")
df = df[df['Sellable Qty.'].astype(float) > 0]

def parse_usd(val):
    try:
        if isinstance(val, str):
            return float(val.replace('$','').replace(',','').strip())
        return float(val) if pd.notnull(val) else 0
    except Exception:
        return 0

def get_peak_usd_price(row):
    try:
        if row['Symbol'] == 'CDNS':
            return 324
        qty = float(row['Sellable Qty.']) if pd.notnull(row['Sellable Qty.']) else 0
        market_val = parse_usd(row['Est. Market Value'])
        return market_val / qty if qty else 0
    except Exception:
        return 0

def get_usd_to_inr(row):
    if row['Symbol'] == 'CDNS' and row['Vest Date'] == '31-JAN-2025':
        return 85.45
    return 83.20

df['Peak USD Price'] = df.apply(get_peak_usd_price, axis=1)
df['USD to INR'] = df.apply(get_usd_to_inr, axis=1)

# ITR2 Schedule FA A3 field mapping as per official schema
df['Country/Region name'] = 'United States'
df['Country Name and Code'] = 'United States (US)'
df['Name of entity'] = "Cadence Design Systems Inc."
df['Address of entity'] = "2655 Seely Avenue San Jose CA"
df['ZIP Code'] = '95134'
df['Nature of entity'] = df['Plan Type']
df['Date of acquiring the interest'] = df['Vest Date']

# Initial value: FMV at acquisition (Grant Date FMV or Purchase Date FMV × qty × FX rate)
def get_initial_value(row):
    try:
        qty = float(row['Sellable Qty.']) if pd.notnull(row['Sellable Qty.']) else 0
        usd_inr = row['USD to INR']
        if pd.notnull(row.get('Grant Date FMV', None)) and parse_usd(row['Grant Date FMV']) > 0:
            return parse_usd(row['Grant Date FMV']) * qty * usd_inr
        if pd.notnull(row.get('Purchase Date FMV', None)) and parse_usd(row['Purchase Date FMV']) > 0:
            return parse_usd(row['Purchase Date FMV']) * qty * usd_inr
        return parse_usd(row['Est. Market Value']) * usd_inr
    except Exception:
        return 0

df['Initial value of the investment'] = df.apply(get_initial_value, axis=1)

# Peak value: highest price × qty × FX rate
df['Peak value of investment during the Period'] = df['Peak USD Price'] * df['Sellable Qty.'].astype(float) * df['USD to INR']

# Closing balance: value at end of period (Est. Market Value × FX rate)
df['Closing balance'] = df['Est. Market Value'].apply(parse_usd) * df['USD to INR']

# Dividends/credits: usually 0 for ESPP/RSU
df['Total gross amount paid/credited with respect to the holding during the period'] = 0

# Sale/redemption proceeds: 0 if not sold
df['Total gross proceeds from sale or redemption of investment during the period'] = 0

itr2_df = df[[
    'Country/Region name',
    'Country Name and Code',
    'Name of entity',
    'Address of entity',
    'ZIP Code',
    'Nature of entity',
    'Date of acquiring the interest',
    'Initial value of the investment',
    'Peak value of investment during the Period',
    'Closing balance',
    'Total gross amount paid/credited with respect to the holding during the period',
    'Total gross proceeds from sale or redemption of investment during the period'
]]

itr2_df.to_csv("ITR2_Schedule_FA_A3_Etrade.csv", index=False)